# So given the original question: Do states with more places data have different metrics in general? (maybe experience more or less year to year change)


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statistics
import scipy.stats as stats
from distfit import distfit
import statsmodels.api as sm 
import pylab
from numpy import cov
from itertools import permutations 
import itertools
from scipy.stats import spearmanr
from random import sample 
import random 
pd.set_option('display.max_colwidth', None)
from collections import Counter
from tabulate import tabulate

# First the data is read in 
The CSV files are read in using Pandas

In [ ]:
s2020 =pd.read_csv("C:/Users/amcfa/Downloads/PLACES__Local_Data_for_Better_Health__Place_Data_2020_release.csv")
s2021 =pd.read_csv("C:/Users/amcfa/Downloads/PLACES__Local_Data_for_Better_Health__Place_Data_2021_release.csv")
s2022 =pd.read_csv("C:/Users/amcfa/Downloads/PLACES__Local_Data_for_Better_Health__Place_Data_2022_release.csv")


In [ ]:
len(s2022)

In [ ]:
#lis2 = state.loc[state['Measure']==second_element,'Data_Value'] 

# Some Basic Data Cleaning
All NA values are dropped. And all rows and columns have been inspected to ensure they are digits and make sense.The only null values remaining are data value footnotes(Which are NaN) but are of no importance. 

In [ ]:
# Drop NA values
s2020['Measure'] = s2020['Measure'].str.replace(" ", "_") # Strip whitespace and replace with _
s2021['Measure'] = s2021['Measure'].str.replace(" ", "_") # Strip whitespace and replace with _
s2022['Measure'] = s2022['Measure'].str.replace(" ", "_") # Strip whitespace and replace with _
s2020= s2020[s2020['Data_Value'].notna()]
s2021= s2021[s2021['Data_Value'].notna()]
s2022= s2022[s2022['Data_Value'].notna()]

In [ ]:
print(s2020.isnull().sum(),s2021.isnull().sum(),s2022.isnull().sum())

In [ ]:
a = s2020['Data_Value'].describe()
b = s2021['Data_Value'].describe()
c = s2022['Data_Value'].describe()
print((a,b,c))

Get the correlations of Measure by measure per state. 
Step 1) Get measure and states. This will end up really big. 
The below code compiles everything by state and test. Type the name of the state (2 letters) below and a big dataframe will appear. 
I can use that to run mass correlation analysis. 
While Exec is not recommended. For this project with 153 different variables, each having 30 different possible measures, it can become extremely unwieldly very fast. 


In [ ]:

#Getting measures by themselves and creating just a ton of df
sa=s2020['StateAbbr'].unique()  # Getting all names of all sates
n = len(sa)
new_list_of_titles = []
dfs = []
for i in sa:
	if i not in new_list_of_titles:
		new_list_of_titles.append(i)
print(new_list_of_titles)


grouped = s2020.groupby(s2020['StateAbbr']) # Group by year
for i in new_list_of_titles:
	exec(f'{i}=grouped.get_group("{i}")')  # Using f string here to execute code. Getting the groups of the grouped well.groups. and executing based on state. So I make 51 dataframes this way one for each
dataframes_dict = {}

# Create DataFrames and store them in the dictionary 
for i in new_list_of_titles:
	dataframes_dict[i] = eval(i)
# put all of them into a list so I can maybe iterateover them easier Use this for the iteration
#eval(f'dataframes_list_{year} = list(dataframes_dict.values())') # no one sees me using exec. Nope. 
dataframes_list2020 = list(dataframes_dict.values())
	

In [ ]:
	
sa=s2021['StateAbbr'].unique()  # Getting all names of all sates
n = len(sa)
new_list_of_titles = []
dfs = []
for i in sa:
	if i not in new_list_of_titles:
		new_list_of_titles.append(i)
print(new_list_of_titles)


grouped = s2021.groupby(s2021['StateAbbr']) # Group by year
for i in new_list_of_titles:
	exec(f'{i}=grouped.get_group("{i}")')  # Using f string here to execute code. Getting the groups of the grouped well.groups. and executing based on state. So I make 51 dataframes this way one for each
dataframes_dict = {}

# Create DataFrames and store them in the dictionary 
for i in new_list_of_titles:
	dataframes_dict[i] = eval(i)
# put all of them into a list so I can maybe iterateover them easier Use this for the iteration
#eval(f'dataframes_list_{year} = list(dataframes_dict.values())') # no one sees me using exec. Nope. 
dataframes_list2021 = list(dataframes_dict.values())

In [ ]:
# Getting measures by themselves and creating just a ton of df
sa=s2022['StateAbbr'].unique()  # Getting all names of all sates
n = len(sa)
new_list_of_titles = []
dfs = []
for i in sa:
	if i not in new_list_of_titles:
		new_list_of_titles.append(i)
print(new_list_of_titles)


grouped = s2022.groupby(s2022['StateAbbr']) # Group by year
for i in new_list_of_titles:
	exec(f'{i}=grouped.get_group("{i}")')  # Using f string here to execute code. Getting the groups of the grouped well.groups. and executing based on state. So I make 51 dataframes this way one for each
dataframes_dict = {}

# Create DataFrames and store them in the dictionary 
for i in new_list_of_titles:
	dataframes_dict[i] = eval(i)
# put all of them into a list so I can maybe iterateover them easier Use this for the iteration
#eval(f'dataframes_list_{year} = list(dataframes_dict.values())') # no one sees me using exec. Nope. 
dataframes_list2022 = list(dataframes_dict.values())


In [ ]:
dataframes_list2020[0].head(1)

In [ ]:
dataframes_list2021[0].head(1)

In [ ]:
dataframes_list2022[0].head(1)

All Cells seem normal. 

# Workshop Code as of 11/08/2023 - this code works (For 2020)
Lets Hope This code works right, and it does At least it is correct otherwise(I Checked for unique Corr values and there were almost as many as the length of the list)
I needed correlation Values for each set

In [ ]:
x=[]
y=[]
n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())		
		print("Ahhh")

In [ ]:
x=[]
y=[]

n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2020[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
	for i in intersection_list:
		y.append([i])
		lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
		lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
		if len(lis1) != len(lis2):
			min_len = (min(len(lis1), len(lis2)))
			#print(min_len)
			if len(lis1) == min_len:
		
				lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				#print(lis2)
			if len(lis2) == min_len:
				lis1= random.sample(list(lis1), min_len)
			else:
				continue
		#print(stats.spearmanr(lis1, lis2),i)
		x.append([subsetb,subsetc,stats.spearmanr(lis1, lis2),i])
		print_count += 1
		#print("Total prints:", print_count)
		result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df1 = pd.DataFrame(x)


In [ ]:
df1

In [ ]:
df1['state'] = df1[0]+df1[1]
df1['state'] = df1['state'].astype(str)
df1['result_string'] = df1[2].astype(str) # gotta change the kurkwalis to string
df1['Corr'] = df1['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df1['p-value'] = df1['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df1['State1'] = df1[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df1['State2'] = df1[1].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df1= df1.drop(['result_string',0,1,2],axis=1)
df1['p-value'] = df1['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df1['p-value'] = pd.to_numeric(df1['p-value'], errors='coerce') # Convert P value to numeric
df1['Measure1'] = df1[3]
empty_rows = df1[df1.isna().any(axis=1)]
df1 = df1[~df1.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 

# So this Value below can be used to select all of the state combinations that pass correlation test 11/15/2023

In [ ]:
filtered_df = df1[df1['Corr'] < 0.5]
filtered_df1 = filtered_df[filtered_df["Corr"]>-0.5]


In [ ]:
grouped = filtered_df1.groupby('State2') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')

In [ ]:

n = len(dataframes_list2021)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2021 = dataframes_list2021[j] # matches 2020 is first dataframe
	subseta=(matches_2021['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2021[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())		
		print("Ahhh")

In [ ]:
x1=[]
y=[]

n = len(dataframes_list2021)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2021 = dataframes_list2021[j] # matches 2020 is first dataframe
	subseta=(matches_2021['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2021[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2021[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
	for i in intersection_list:
		y.append([i])
		lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
		lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
		if len(lis1) != len(lis2):
			min_len = (min(len(lis1), len(lis2)))
			#print(min_len)
			if len(lis1) == min_len:
		
				lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				#print(lis2)
			if len(lis2) == min_len:
				lis1= random.sample(list(lis1), min_len)
			else:
				continue
		#print(stats.spearmanr(lis1, lis2),i)
		x1.append([subsetb,subsetc,stats.spearmanr(lis1, lis2),i])
		print_count += 1
		#print("Total prints:", print_count)
		#result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together


In [ ]:
df2 = pd.DataFrame(x1)
df2

In [ ]:
df2['state'] = df2[0]+df2[1]
df2['state'] = df2['state'].astype(str)
df2['result_string'] = df2[2].astype(str) # gotta change the kurkwalis to string
df2['Corr'] = df2['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df2['p-value'] = df2['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df2['State1'] = df2[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df2['State2'] = df2[1].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df2= df2.drop(['result_string',0,1,2],axis=1)
df2['p-value'] = df2['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df2['p-value'] = pd.to_numeric(df2['p-value'], errors='coerce') # Convert P value to numeric
df2['Measure1'] = df2[3]


In [ ]:
filtered_df = df2[df2['Corr'] < 0.5]
filtered_df2 = filtered_df[filtered_df["Corr"]>-0.5]

In [ ]:
grouped = filtered_df2.groupby('State2') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')

In [ ]:

n = len(dataframes_list2022)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2022 = dataframes_list2022[j] # matches 2020 is first dataframe
	subseta=(matches_2022['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2022[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())		
		print("Ahhh")	


In [ ]:
x2=[]
y=[]

n = len(dataframes_list2022)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2022 = dataframes_list2022[j] # matches 2020 is first dataframe
	subseta=(matches_2022['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2022[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2022[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
	for i in intersection_list:
		y.append([i])
		lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
		lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
		if len(lis1) != len(lis2):
			min_len = (min(len(lis1), len(lis2)))
			#print(min_len)
			if len(lis1) == min_len:
		
				lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				#print(lis2)
			if len(lis2) == min_len:
				lis1= random.sample(list(lis1), min_len)
			else:
				continue
		#print(stats.spearmanr(lis1, lis2),i)
		x2.append([subsetb,subsetc,stats.spearmanr(lis1, lis2),i])
		print_count += 1
		#print("Total prints:", print_count)
		#result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df3 = pd.DataFrame(x2)
df3

In [ ]:
df3['state'] = df3[0]+df3[1]
df3['state'] = df3['state'].astype(str)
df3['result_string'] = df3[2].astype(str) # gotta change the kurkwalis to string
df3['Corr'] = df3['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df3['p-value'] = df3['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df3['State1'] = df3[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df3['State2'] = df3[1].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df3= df3.drop(['result_string',0,1,2],axis=1)
df3['p-value'] = df3['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df3['p-value'] = pd.to_numeric(df3['p-value'], errors='coerce') # Convert P value to numeric
df3['Measure1'] = df3[3]

In [ ]:
filtered_df = df3[df3['Corr'] < 0.5]
filtered_df3 = filtered_df[filtered_df["Corr"]>-0.5]

In [ ]:
grouped = filtered_df3.groupby('State2') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')

# After data is collected it can now be used for visualization and such (11/08/2023) Also not needed

Well I can now see if states with higher counts of PLACES data have different correlation values. Why not. First I need to check for some empty values. And there were just a few. 



df1 = pd.DataFrame(result1)
df3 = pd.DataFrame(result2)
df5 = pd.DataFrame(result3)

# I had mixed data in the column so I needed to weirdly split this thing up 
def split_column(row):  
    return pd.Series([row[0][0], row[1]])  
df1[['state', 'Corr']] = df1[0].apply(split_column)
df1.drop(0, axis=1, inplace=True) # Drop the df in place. dont assign to another var
df1[['Measure1','Measure2']] = df1[1].apply(lambda x: pd.Series(x)) # This splits the string up for us and such
empty_cells = df1.isna() # Are there any na values
empty_cells_count = empty_cells.sum()
print(empty_cells_count)
empty_rows = df1[df1.isna().any(axis=1)]
df2 = df1[~df1.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
df2.drop(1,axis=1, inplace=True)
len(df2.Corr.unique())

# I had mixed data in the column so I needed to weirdly split this thing up 
def split_column(row):  
    return pd.Series([row[0][0], row[1]])  
df3[['state', 'Corr']] = df3[0].apply(split_column)
df3.drop(0, axis=1, inplace=True) # Drop the df in place. dont assign to another var
df3[['Measure1','Measure2']] = df3[1].apply(lambda x: pd.Series(x)) # This splits the string up for us and such
empty_cells = df3.isna() # Are there any na values
empty_cells_count = empty_cells.sum()
print(empty_cells_count)
empty_rows = df3[df3.isna().any(axis=1)]
df4 = df3[~df3.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
df4.drop(1,axis=1, inplace=True)
len(df4.Corr.unique())

# I had mixed data in the column so I needed to weirdly split this thing up 
def split_column(row):  
    return pd.Series([row[0][0], row[1]])  
df5[['state', 'Corr']] = df5[0].apply(split_column)
df5.drop(0, axis=1, inplace=True) # Drop the df in place. dont assign to another var
df5[['Measure1','Measure2']] = df5[1].apply(lambda x: pd.Series(x)) # This splits the string up for us and such
empty_cells = df5.isna() # Are there any na values
empty_cells_count = empty_cells.sum()
print(empty_cells_count)
empty_rows = df5[df5.isna().any(axis=1)]
df6 = df5[~df5.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
df6.drop(1,axis=1, inplace=True)
len(df6.Corr.unique())

## 11/08/2023
Maybe the first correlation Value I can look at is the overall max cor value for each state

# a cool little form of df groupby kind of neat. Uses idxmax, which returns the max of each index
def max_corr(group):
    max_corr_idx = group['Corr'].idxmax()
    return group.loc[max_corr_idx,['Corr','Measure1','Measure2']] 

result1 = df2.groupby('state').apply(max_corr).reset_index()  # Groupby state
result2 = df4.groupby('state').apply(max_corr).reset_index()  # Groupby state
result3 = df6.groupby('state').apply(max_corr).reset_index()  # Groupby state

# 11/08/2023

So viewing the results briefly, Chronic kidney disease and stroke have some correlation in this data set. And I notice the correlation values for the other two years are much much lower. It is strange. 

# So given the original question: Do states with more places data have higher metrics in general?

Below are the counts for all of the places data divided by state, PA, TX, CA, IL, have the most by a wide margin so let's look at those first.

In [ ]:
m=s2020[s2020['StateAbbr']=='NC']
m['LocationName'].nunique()

In [ ]:
# Data
years = ['2020', '2021', '2022']
values = [28329, 28329, 28329]


plt.bar(years, values, color='green')

plt.xlabel('Year')
plt.ylabel('Values')
plt.title('Sum of all cities in PLACES Dataset per year')


plt.show()

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].nunique()
city_counts1 = city_counts1.sort_values(ascending=False)
city_counts2 = s2021.groupby('StateAbbr')['LocationName'].nunique()
city_counts2 = city_counts2.sort_values(ascending=False)
city_counts3 = s2022.groupby('StateAbbr')['LocationName'].nunique()
city_counts3 = city_counts3.sort_values(ascending=False)

In [ ]:
city_counts1['CA']

In [ ]:
city_counts1['PA']

In [ ]:
city_counts1['TX']

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts1,palette="dark:#5A9_r")


ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts2,palette="dark:#5A9_r")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2021")

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts3,palette="dark:#5A9_r")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2022")

# No matter what Use the cor analysis to select the rows or columns (whatever) that have little or no correlation and run statistical analysis 
Year to year there is a decrease in correlation between variables. The first year there is a wide range of correlation values, and the following years there is almost none. 

And looking at mean and median values for correlations for each year and set of states, all show a similiar decrease in correlation values. 

# Correlations seem fairly similiar. ## Big Blob from the bottom where correlation analysis was done will go here

Now time to look at actual data itself. 

In [ ]:
df2

In [ ]:
df1

In [ ]:
abcd = df1[df1['Corr'] < 0.5]
defg=abcd[abcd['Corr'] >-0.5]

city_counts1 =defg.groupby('state')['Measure1'].nunique()

In [ ]:
df1['state']

In [ ]:
abcd = df1[df1['Corr'] < 0.5]
defg=abcd[abcd['Corr'] >-0.5]

city_counts1 =defg.groupby('state')['Measure1'].nunique()
cleaned_df = df1[df1.index.isin(defg.index)]
city_counts2 =cleaned_df.groupby('state')['Measure1'].nunique()
ah1 = df1[df1['Corr'] > 0.5]
len(ah1['Measure1'].unique())



In [ ]:
cleaned_df

In [ ]:
defg

In [ ]:
## Working down here for correlations as of 11/15/2023

#Trying to organize my values and decide if the values should be dropped. The below code filtered out all of the combinations that had corr values that were not significant. As far as how to move forward. Not sure
# I had mixed data in the column so I needed to weirdly split this thing up 


abcd = df2[df2['Corr'] < 0.5]
defg=abcd[abcd['Corr'] >-0.5]

city_counts1 =defg.groupby('state')['Measure1'].nunique()
cleaned_df2 = df2[df2.index.isin(defg.index)]
city_counts2 =cleaned_df2.groupby('state')['Measure1'].nunique()
ah2 = df2[df2['Corr'] > 0.5]
len(ah2['Measure1'].unique())



# Correlation values are good

len(cleaned_df2)

In [ ]:
abcd = df3[df3['Corr'] < 0.5]
defg=abcd[abcd['Corr'] >-0.5]

city_counts3 =defg.groupby('state')['Measure1'].nunique()
cleaned_df3 = df3[df3.index.isin(defg.index)]
city_counts3 =cleaned_df3.groupby('state')['Measure1'].nunique()
ah3 = df3[df3['Corr'] > 0.5]
len(ah3['Measure1'].unique())
len(cleaned_df3)


In [ ]:
'''defg['Measure2'].value_counts() # Sorting in place

df3=df2.sort_values(by=['Corr'])
df3.sort_values(['Measure1', 'Measure2'], inplace=True) # Obtaining measures
mean_values = df3['Corr'].mean

df3['Combined'] = df3['Measure1'].astype(str) + '_' + df3['Measure2'].astype(str) # Combining strings
grouped_df = df3.groupby('Combined')
mean_values = grouped_df['Corr'].median()
mean_values '''



In [ ]:
'''mean_values_reset = mean_values.reset_index()
mean_values_reset 
filtered_series = mean_values[(mean_values < 0.5) & (mean_values > -0.5)] # Filter all of the values. 
filtered_series=filtered_series.reset_index()
stuff_to_drop=(mean_values_reset[~mean_values_reset.Combined.isin(filtered_series.Combined)])
print(stuff_to_drop['Combined'])
stuff_to_keep=(df3[~df3.Combined.isin(stuff_to_drop.Combined)])
stuff_to_keep['Combined'].nunique()
grouped = stuff_to_keep.groupby('state') # So group each by state '''

In [ ]:
'''fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed
for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')'''

## Big blob for correlation analysis will stop here

# The below code tests for normality using a D’Agostino’s K^2 test. 

But just because the data is not normally distributed, does not mean it is Gaussian in shape

In [ ]:
from scipy.stats import normaltest
alpha = 0.05
state= dataframes_list2020[0]
subset1=(state['Measure'].unique())
for i in subset1:
	lis1 = state.loc[state['Measure']==i,'Data_Value']
	stat, p = normaltest(lis1)
	print('Statistics=%.3f, p=%.3f' % (stat, p))
	if p > alpha:
 		print('Sample looks Gaussian (fail to reject H0)')
	else:
 		print('Sample is not Gaussian, maybe normal (reject H0)')
	
  

# Originally the data was processed using the fitdist function.
In order to define the best fit distribution of all data, to see which statistical test to employ for each state, and metric for that state. 

And Now I am going to use R to help me work with this data. R has great statistical packages to work with just means of data for example. Also there are few sets of data that fit into a normal distribution so I will use nonparametric tests. 

df1= pd.read_csv('2020_data_df')  # Read in the data
df2= pd.read_csv('2021_data_df')
df3= pd.read_csv('2022_data_df')
trying2020 = df1[['State','Measure_Name','Best_Fit_Distribution','mean']] # Select only this subset
trying2021 = df2[['State','Measure_Name','Best_Fit_Distribution','mean']]
trying2022 = df3[['State','Measure_Name','Best_Fit_Distribution','mean']]
#norm2020 = trying2020[trying2020['Best_Fit_Distribution']=='norm'] # Further Filter each DF
#norm2021 = trying2021[trying2021['Best_Fit_Distribution']=='norm']
#norm2022 = trying2022[trying2022['Best_Fit_Distribution']=='norm']

In [ ]:
#trying2020 =trying2020[~trying2020['Best_Fit_Distribution'].str.contains('norm')]

In [ ]:
#trying2021 =trying2021[~trying2021['Best_Fit_Distribution'].str.contains('norm')]
#trying2022 =trying2022[~trying2022['Best_Fit_Distribution'].str.contains('norm')]

Any normally distributed data I can test using the paired t test
to start maybe split all of the dat on these lines

trying2020[['group', 'year', 'testnumber']] = trying2020['State'].apply(lambda x: pd.Series(str(x).split('_'))) # apply seems to force the data to split
trying2021[['group', 'year', 'testnumber']] = trying2021['State'].apply(lambda x: pd.Series(str(x).split('_')))
trying2022[['group', 'year', 'testnumber']] = trying2022['State'].apply(lambda x: pd.Series(str(x).split('_')))

# 11/09/2023
Now group again by state and measure. grouped list of df,then group by measure and then compare year to year. And this is difficult because I cant just merge the DF because of their size. I would run into a memory error. 


In [ ]:
s2020.head(1)

The below code is trying to go by the dataframes in place kind of. It finds the first match, then matches in the others. While it is not as fast it should hopefully not be super computationally expensive.

## Working On code down here 11/09/2023 The code below is for all Data 

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs. Post hoc comparisons between groups are required to determine which groups are different.*

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html


For the code below. I take each state index, match the same states in the other dataframes and then run all stats kruskal tests on the variables. This test can also take in arrays of different lengths so I commented out the random sampling to make them the same. 

I need to see if the medians of the groups are rejected more for the cities with more places data than others. Over years. 

## As of 11/15/2023 worked here on excluding highly correlated values

# The below shortens the result dataframe.

The above gives all results where there are no year to year changes. And it seems all states change year to year when compared to themselves. So How about just the biggest cities states tested for differences against all other states. 

# 11/09/20203 The below code Compares texas to all other states for all metrics for the year 2020. 
If my lists are not the same length I need to do some repetition and take the averages no?

In [ ]:
x=[]
y=[]
n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())		
		print("Ahhh")

In [ ]:
x=[]
y=[]

n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2020[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
	for i in intersection_list:
		y.append([i])
		lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
		lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
		#if len(lis1) != len(lis2):
			#min_len = (min(len(lis1), len(lis2)))
			#print(min_len)
			#if len(lis1) == min_len:
		
				#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				#print(lis2)
			#if len(lis2) == min_len:
				#lis1= random.sample(list(lis1), min_len)
			#else:
				#continue
		#print(stats.kruskal(lis1, lis2),i)
		x.append([subsetb,subsetc,stats.kruskal(lis1, lis2),i])
		print_count += 1
		#print("Total prints:", print_count)
		result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df1=pd.DataFrame(result1)

In [ ]:
df1.head(2)

In [ ]:

df1['result_string'] = df1[0].astype(str) # gotta change the kurkwalis to string
df1['statistic'] = df1['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df1['p-value'] = df1['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df1['State'] = df1[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df1['state'] = df1[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ''.join(x[0]))
df1[['State1','State2']]=df1['State'].apply(lambda x: pd.Series(str(x).split(',')))
df1= df1.drop(['result_string',0],axis=1)
df1['p-value'] = df1['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df1['p-value'] = pd.to_numeric(df1['p-value'], errors='coerce') # Convert P value to numeric
#filtered_df = df2[df2['p-value'] < 0.05]
df1['Measure']= df1[1]

In [ ]:
len(df1)

In [ ]:
len(cleaned_df)

In [ ]:

result_2020 = df1.loc[cleaned_df.index]
result_2020

# 2020 texas code stops


# 2021 texas code starts

In [ ]:
x=[]
y=[]
n = len(dataframes_list2021)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2021 = dataframes_list2021[j] # matches 2021 is first dataframe
	subseta=(matches_2021['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2021[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	

In [ ]:
x=[]
y=[]
n = len(dataframes_list2021)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2021 = dataframes_list2021[j] # matches 2021 is first dataframe
	subseta=(matches_2021['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2021[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2021[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
		for i in intersection_list:
			y.append([i])
			lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
			lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
			#if len(lis1) != len(lis2):
				#min_len = (min(len(lis1), len(lis2)))
				#print(min_len)
				#if len(lis1) == min_len:
			
					#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
					#print(lis2)
				#if len(lis2) == min_len:
					#lis1= random.sample(list(lis1), min_len)
				#else:
					#continue
			#print(stats.kruskal(lis1, lis2),i)
			x.append([subsetb,subsetc,stats.kruskal(lis1, lis2),i])
			print_count += 1
			#print("Total prints:", print_count)
			result2 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df3=pd.DataFrame(result2)

In [ ]:
df3

In [ ]:
df3['result_string'] = df3[0].astype(str) # gotta change the kurkwalis to string
df3['statistic'] = df3['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df3['p-value'] = df3['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df3['State'] = df3[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df3['state'] = df3[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ''.join(x[0]))
df3[['State1','State2']]=df3['State'].apply(lambda x: pd.Series(str(x).split(',')))
df3= df3.drop(['result_string',0],axis=1)
df3['p-value'] = df3['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df3['p-value'] = pd.to_numeric(df3['p-value'], errors='coerce') # Convert P value to numeric
df3['Measure']= df3[1]

In [ ]:
df3

In [ ]:
df3['Measure'] = df3[1].astype(str)
df3['Measure'] = df3['Measure'].str.replace("['", '').str.replace("']", '')
df3= df3.drop([1],axis=1)

In [ ]:
df3

In [ ]:

filtered_df2['Measure'] = filtered_df2[3].astype(str)
filtered_df2['Measure'] = filtered_df2['Measure'].str.replace("['", '').str.replace("']", '')
filtered_df2= filtered_df2.drop([3],axis=1)



In [ ]:
filtered_df2['state'] = filtered_df2['state'].astype(str)
filtered_df2['state'] = filtered_df2['state'].str.replace("['", '').str.replace("']", '')

In [ ]:
filtered_df2

In [ ]:
results_2021 = df3.merge(filtered_df2, how='inner', on=['Measure','state'])


In [ ]:
filtered_df2.head(2)

In [ ]:
len(df3)

In [ ]:
len(results_2021)

# End of 2021 texas

# Start  of 2022 texas

In [ ]:
x=[]
y=[]
n = len(dataframes_list2022)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2022 = dataframes_list2022[j] # matches 2021 is first dataframe
	subseta=(matches_2022['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2022[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())

In [ ]:
x=[]
y=[]
n = len(dataframes_list2022)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2022 = dataframes_list2022[j] # matches 2021 is first dataframe
	subseta=(matches_2022['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2022[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2022[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
		for i in intersection_list:
			y.append([i])
			lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
			lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
			#if len(lis1) != len(lis2):
				#min_len = (min(len(lis1), len(lis2)))
				#print(min_len)
				#if len(lis1) == min_len:
			
					#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
					#print(lis2)
				#if len(lis2) == min_len:
					#lis1= random.sample(list(lis1), min_len)
				#else:
					#continue
			#print(stats.kruskal(lis1, lis2),i)
			x.append([subsetb,subsetc,stats.kruskal(lis1, lis2),i])
			print_count += 1
			#print("Total prints:", print_count)
			result3 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df4=pd.DataFrame(result3)
len(df4)

In [ ]:
df4['result_string'] = df4[0].astype(str) # gotta change the kurkwalis to string
df4['statistic'] = df4['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df4['p-value'] = df4['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df4['State'] = df4[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df4['state'] = df4[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ''.join(x[0]))
df4[['State1','State2']]=df4['State'].apply(lambda x: pd.Series(str(x).split(',')))
df4= df4.drop(['result_string',0],axis=1)
df4['p-value'] = df4['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df4['p-value'] = pd.to_numeric(df4['p-value'], errors='coerce') # Convert P value to numeric


In [ ]:
df4

In [ ]:
df4['Measure'] = df4[1].astype(str)
df4['Measure'] = df4['Measure'].str.replace("['", '').str.replace("']", '')
df4= df4.drop([1],axis=1)

In [ ]:
filtered_df3

In [ ]:
filtered_df3= filtered_df3.rename(columns={3:"Measure"})
filtered_df3['state'] = filtered_df3['state'].astype(str)
filtered_df3['state'] = filtered_df3['state'].str.replace("['", '').str.replace("']", '')
filtered_df3['Measure'] = filtered_df3['Measure'].str.strip()
filtered_df3['state'] = filtered_df3['state'].str.strip()
df4["Measure"] = df4["Measure"].str.strip()
df4["state"] = df4["state"].str.strip()

In [ ]:
filtered_df3

In [ ]:
result_2022 = df4.merge(filtered_df3, how='inner', on=['Measure','state'])
result_2022

In [ ]:
len(df4)

# 11/10/2023
So to see if Texas for example differed more than any other state, I need to sum all of the times texas occurs and then sum all of the other states. The above filtered DF is where there is a difference from the median. And if texas occurs more, then.. it occurs more. It does not answer the question on if the data is higher or lower. Just that a difference exists.

In [ ]:
len(df4)

In [ ]:
from collections import Counter
print(Counter(filtered_df['State1']))

In [ ]:
from collections import Counter
print(Counter(filtered_df['State1']))

In [ ]:
from collections import Counter
print(Counter(filtered_df['State2']))

In [ ]:
from collections import Counter
print(Counter(filtered_df1['State2']))

Since all of the results are overwhelmingly, yes the values are different. Post Hoc analysis must be performed.The dunn

# Start of the Dunn test 

Since the data are mostly nonparametric. I can only say if they are different from each other, and not whether one is higher or lower. But if the sample sizes are large enough normality might be able to be assumed. 

In [ ]:
x=[]
y=[]
n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())		
		print("Ahhh")

In [ ]:
import scikit_posthocs as sp

# 11/10/2023
Okay I will now Work through each DF, and gather all of the data I need. My goal is to waste as little space as possible. 

In [ ]:
list_of_measures1 = s2020['Measure'].unique().tolist() # Put all unique values in a list

In [ ]:
list_of_measures1

In [ ]:
ugh = [] # New  Empty List
state_names = []

list_of_measures1 = s2020['Measure'].unique().tolist() # Put all unique values in a list


result1 = s2020[s2020['Measure'] == 'Cancer_(excluding_skin_cancer)_among_adults_aged_>=18_years'] # Go over each measure
for i in new_list_of_titles:
	condition = lambda x: x['StateAbbr'] == i  # Gather all of the state data
	filtered_data = result1[result1.apply(condition, axis=1)] # Filter all of the state data
	lis1 = filtered_data['Data_Value'].tolist()
	ugh.append(lis1)
	state_names.append(i)
ab= sp.posthoc_dunn(ugh, p_adjust = 'bonferroni')
df_dunn_result = pd.DataFrame(ab, index=ab.index, columns=ab.columns) # in order to get the data to appear appropriately, need to include index and columns
filtered_df5=df_dunn_result.where(df_dunn_result<0.05)
indices = [(l, m) for l, row in enumerate(filtered_df5.values) for m, value in enumerate(row) if pd.notna(value)]
	

In [ ]:
state_mapping = {index: state_names[i] for i, index in enumerate(range(len(state_names)))} # Enumerates the state names list
state_pairs = [(state_mapping[i], state_mapping[j]) for i, j in indices] # Maps the state names listto its respective index. 
sorting_pairs = set([tuple(sorted(pair)) for pair in state_pairs]) # Okay so sort the things and make them into a set. 

In [ ]:
only_dunn_TX=(df_dunn_result[43].where(df_dunn_result[43]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_TX=only_dunn_TX.dropna()
len(only_dunn_TX)
only_dunn_CA=(df_dunn_result[4].where(df_dunn_result[4]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_CA=only_dunn_CA.dropna()
len(only_dunn_CA)
only_dunn_PA=(df_dunn_result[38].where(df_dunn_result[38]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_PA=only_dunn_PA.dropna()
len(only_dunn_PA)
only_dunn_IL=(df_dunn_result[12].where(df_dunn_result[12]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_IL=only_dunn_IL.dropna()
len(only_dunn_IL)
only_dunn_SD=(df_dunn_result[41].where(df_dunn_result[41]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_SD=only_dunn_SD.dropna()
len(only_dunn_SD)
only_dunn_WY=(df_dunn_result[50].where(df_dunn_result[50]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_WY=only_dunn_WY.dropna()
len(only_dunn_WY)

In [ ]:
len(only_dunn_TX)

# 11/11/2023 Sorted all of the state counts
So using the above code and visualizing it. It is easy to see that Over all Texas

In [ ]:
print(len(set(indices)))

In [ ]:
# Have to count rows and columns separately. 
rows, columns = zip(*indices)

# Count occurrences of each row and column index
row_counts = Counter(rows)
column_counts = Counter(columns)
dataframe_for_rows=pd.DataFrame.from_dict(row_counts, orient='index', columns=['Count'])
dataframe_for_rows['States']= [state_names[row] for row in dataframe_for_rows.index]

In [ ]:
dataframe_for_rows1=pd.DataFrame.from_dict(row_counts, orient='index', columns=['Count'])
# Now I have to calculate the weights


In [ ]:
dataframe_for_rows2 = dataframe_for_rows.set_index(['States'], drop=True)

In [ ]:
hm=dataframe_for_rows2.T
print(hm.to_markdown())

In [ ]:
dataframe_for_rows2['calculated'] = dataframe_for_rows2['Count'].apply(lambda x: 'yes' if x < 46 else 'no')

In [ ]:
len(dataframe_for_rows2[dataframe_for_rows2['calculated']=='yes'])

In [ ]:


fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(hm,palette="cividis")

ax.set(xlabel="States",ylabel ="Count ",title="Locations per State 2020")

Well. I can tell that Texas is often different from the state to which it is compared. But which direction it differs is hard to discern.So maybe it is time to employ some parametric methods after all.

# 11/11/2023
Got all of the state names sorted and counted now need to make the graph look nicer. because it looks very cluttered at the moment. 

In [ ]:
# use the scatterplot function to build the bubble map
sns.scatterplot(data=dataframe_for_rows, x="States", y="Count", size="Count", legend=False, sizes=(20, 200),alpha=0.5)

# show the graph
plt.show()

# 11/12/2023

Going to use the levene test to see if samples come from equal variance distribution

If the resulting p-value of Levene's test is less than some significance level (typically 0.05), the obtained differences in sample variances are unlikely to have occurred based on random sampling from a population with equal variances. Thus, the null hypothesis of equal variances is rejected and it is concluded that there is a difference between the variances in the population. 

https://en.wikipedia.org/wiki/Levene%27s_test

In [ ]:
x=[]
y=[]
n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0

for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2020[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
	for i in intersection_list:
		y.append([i])
		lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
		lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
		#if len(lis1) != len(lis2):
			#min_len = (min(len(lis1), len(lis2)))
			#print(min_len)
			#if len(lis1) == min_len:
		
				#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				#print(lis2)
			#if len(lis2) == min_len:
				#lis1= random.sample(list(lis1), min_len)
			#else:
				#continue
		#print(stats.levene(lis1, lis2),i)
		x.append([subsetb,subsetc,stats.levene(lis1, lis2),i])
		#print_count += 1
		#print("Total prints:", print_count)
		result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df4=pd.DataFrame(result1)

In [ ]:
df4

In [ ]:

df4['result_string'] = df4[0].astype(str) # gotta change the kurkwalis to string
df4['statistic'] = df4['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df4['p-value'] = df4['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df4['State'] = df4[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df4['state'] = df4[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ''.join(x[0]))
df4[['State1','State2']]=df4['State'].apply(lambda x: pd.Series(str(x).split(',')))
df4= df4.drop(['result_string',0],axis=1)
df4['p-value'] = df4['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df4['p-value'] = pd.to_numeric(df4['p-value'], errors='coerce') # Convert P value to numeric
#filtered_df = df4[df4['p-value'] < 0.05]
#filtered_df # are there yer to year changes. This code tells us the counts
#filtered_df1 = df4[df4['p-value'] > 0.05]
#filtered_df1 # are there yer to year changes. This code tells us the counts

In [ ]:
df4['Measure1'] = df4[1].astype(str)
df4['Measure1'] = df4['Measure1'].str.replace("['", '').str.replace("']", '')
df4["Measure1"] = df4["Measure1"].str.strip()
df4["state"] = df4["state"].str.strip()

In [ ]:
filtered_df1['state'] = filtered_df1['state'].str.replace("['", '').str.replace("']", '')
filtered_df1["state"] = filtered_df1["state"].str.strip()
filtered_df1["Measure1"] = filtered_df1["Measure1"].str.strip()

In [ ]:
df4= df4.drop([1],axis=1)

In [ ]:
filtered_df1

In [ ]:

filtered_df1= filtered_df1.drop([3],axis=1)

In [ ]:
filtered_df1

In [ ]:
filtered_df_test1= filtered_df1

In [ ]:
filtered_df_test1

In [ ]:
column_to_move = 'Measure1'

# Pop the column and store it in a variable
column_temp = df4.pop(column_to_move)

# Insert the popped column at the beginning of the DataFrame
df4.insert(0, column_to_move, column_temp)

In [ ]:
column_to_move = 'state'

# Pop the column and store it in a variable
column_temp = df4.pop(column_to_move)

# Insert the popped column at the beginning of the DataFrame
df4.insert(1, column_to_move, column_temp)

In [ ]:
column_to_move = 'Measure1'

# Pop the column and store it in a variable
column_temp = filtered_df_test1.pop(column_to_move)

# Insert the popped column at the beginning of the DataFrame
filtered_df_test1.insert(0, column_to_move, column_temp)



In [ ]:
filtered_df_test1

In [ ]:
filtered_df_test1=filtered_df_test1.drop(['Corr'],axis=1)
filtered_df_test1=filtered_df_test1.drop(['p-value'],axis=1)

In [ ]:
filtered_df_test1=filtered_df_test1.drop(['State1'],axis=1)
filtered_df_test1=filtered_df_test1.drop(['State2'],axis=1)

In [ ]:
duplicates = df4[df4.duplicated(['Measure1','state'])]

In [ ]:
len(duplicates)

In [ ]:
df_no_duplicates = df4.drop_duplicates(['Measure1','state'])

In [ ]:
len(df4)

In [ ]:
len(df_no_duplicates)

In [ ]:
result_2020_levene = df_no_duplicates.merge(filtered_df_test1, how='right', on=['Measure1','state'])
result_2020_levene

# So we have the results of the Levene test for 2020 for texas above


## T test starts Below

In [ ]:
x=[]
y=[]
n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0

for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df

	if subseta ==['TX']:
		matches_texas = dataframes_list2020[j]  # If subset is correct then just set the variable to be the correct thing
		subsetb = (matches_texas['StateAbbr'].unique())
	else:
		not_texas = dataframes_list2020[j] # Otherwise iterate over everything
		subsetc = (not_texas['StateAbbr'].unique())
		subset1=(matches_texas['Measure'].unique()) 
		subset2=(not_texas['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2) # now the intersection of all 2	
		intersection_list = list(intersection_of_measures) 
	for i in intersection_list:
		y.append([i])
		lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
		lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
		#if len(lis1) != len(lis2):
			#min_len = (min(len(lis1), len(lis2)))
			#print(min_len)
			#if len(lis1) == min_len:
		
				#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				#print(lis2)
			#if len(lis2) == min_len:
				#lis1= random.sample(list(lis1), min_len)
			#else:
				#continue
		#print(stats.ttest_ind(lis1, lis2),i)
		x.append([subsetb,subsetc,stats.ttest_ind(lis1, lis2),i])
		print_count += 1
		#print("Total prints:", print_count)
		result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df4=pd.DataFrame(result1)
df4['result_string'] = df4[0].astype(str) # gotta change the kurkwalis to string
df4['statistic'] = df4['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
df4['p-value'] = df4['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df4['State'] = df4[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ', '.join(x[0]))
df4[['State1','State2']]=df4['State'].apply(lambda x: pd.Series(str(x).split(',')))
df4= df4.drop(['result_string',0],axis=1)
df4['p-value'] = df4['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
df4['p-value'] = pd.to_numeric(df4['p-value'], errors='coerce') # Convert P value to numeric
#filtered_df = df4[df4['p-value'] < 0.05]
#filtered_df # are there yer to year changes. This code tells us the counts
#filtered_df1 = df4[df4['p-value'] > 0.05]
#filtered_df1 # are there yer to year changes. This code tells us the counts

In [ ]:
s2020.head(1)

## On Moday work on comparing all of the T tests for everything  (11/13/2023)
# Started on it 11/14/2023
To do *If texas and state in lis 1 and lis 2 match AND the measure for filtered_df1. Then Run the equal variance of the scipy.stats.t test
Otherwise run unequal variances for the T test


* Idea Maybe count all the times all of the states were 'significant' and then put that on a graph by state 


* Filtering data by measures and then using that to run either equal variance or unequal t test

* And no matter what make sure to do 'greater' because you want to compare mean 1 and mean 2. With mean1 being the state in question. 


In [ ]:

result_2020_levene["State1"] = result_2020_levene["State1"].str.strip()
result_2020_levene["State2"] = result_2020_levene["State2"].str.strip()
result_2020_levene["State2"] = result_2020_levene["State2"].str.strip()
result_2020_levene['Measure1'] = result_2020_levene["Measure1"].str.strip()
result_2020_levene['Measure'] = result_2020_levene["Measure1"]

In [ ]:
abcd = result_2020_levene[result_2020_levene['p-value'] < 0.05] # For levenes test lower p values mean unequal variance
defg=result_2020_levene[result_2020_levene['p-value'] >0.05 ] #Equal variance

#Levene_test_2020 =defg.groupby('state')['Measure1'].nunique()#
#unequal_variances = df2[df2.index.isin(defg.index)]#

Not sure if we need this below code

In [ ]:
x=[]
y=[]
condition1 = s2020.loc[s2020['StateAbbr']=="TX"]
lis1 = condition1.loc[condition1['Measure']=='Physical_health_not_good_for_>=14_days_among_adults_aged_>=18_years','Data_Value'] # For a dataframe .loc seems to be the simplest for performing this work
condition2 = s2020.loc[s2020['StateAbbr']=='NC']
lis2 = condition2.loc[condition2['Measure']=='Physical_health_not_good_for_>=14_days_among_adults_aged_>=18_years','Data_Value']

if len(lis1) != len(lis2):
	min_len = (min(len(lis1), len(lis2)))
	print(min_len)
	if len(lis1) == min_len:

		lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
		print(lis2)
	if len(lis2) == min_len:
		lis1= random.sample(list(lis1), min_len)
print(stats.ttest_ind(lis1, lis2,equal_var=False),i)
x.append([subsetb,subsetc,stats.ttest_ind(lis1, lis2),i])
print_count += 1
print("Total prints:", print_count)
result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

not sure iif we need the code above

# The below code works at least for now as of 11/14/2023
It runs the T test on the full dataframe, matching to the reduced dataframe. And Runs the T test. Printing the T test statistic, appending everything to a list. 

* If the calculated p-value is below the threshold chosen for statistical significance (usually the 0.10, the 0.05, or 0.01 level), then the null hypothesis is rejected in favor of the alternative hypothesis.

## T test for equal variances for 2020


In [ ]:
defg.head(2)

In [ ]:
# First get dataframe values in a list
prints=defg[['Measure','State1','State2']]
n = len(prints)
current = prints.values.tolist()


In [ ]:
s2020['Measure'] = s2020['Measure'].str.strip()

In [ ]:
x=[]
y=[]
n = len(prints)
for i in range(n):  # For i in range n, set the current list parts to separate variables.
	current1 = current[i]
	measure=(current1[0])

	state1=(current1[1]) # first second, third element
	state2=(current1[2].strip()) # Need to strip random whitesapce


	condition1 = s2020.loc[s2020['StateAbbr']==state1]
	#print(len(condition1))
	lis1 = condition1.loc[condition1['Measure']==measure,'Data_Value'] # For a dataframe .loc seems to be the simplest for performing this work
	condition2 = s2020.loc[s2020['StateAbbr']==state2]
	#print(len(condition2))
	lis2 = condition2.loc[condition2['Measure']==measure,'Data_Value']
	y.append([state1,state2])
	if len(lis1) != len(lis2):
		min_len = (min(len(lis1), len(lis2)))
		
		if len(lis1) == min_len:

			lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
			
		if len(lis2) == min_len:
			lis1= random.sample(list(lis1), min_len)
		#print(len(lis1),len(lis2))
	#print(stats.ttest_ind(lis1, lis2,equal_var=True,alternative='greater'),i)
	x.append([state1,state2,measure,stats.ttest_ind(lis1, lis2,equal_var=True,alternative='greater'),i]) # Make sure to set the alternative to greater
	print_count += 1
	#print("Total prints:", print_count)
	#result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together


In [ ]:
df=pd.DataFrame(x)

In [ ]:
df

In [ ]:

df['result_string'] = df[3].astype(str)
df['t-statistic'] = df['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Extract out all of the relevant p values and t statistics
df['p-value'] = df['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df['State1']= df[0]
df['state2']=df[1]
df= df.drop(['result_string',0,1,3,4],axis=1) 

In [ ]:
df.head(2)

In [ ]:
t_test_equal_var_p_gr = df[df['p-value'] > 0.05] # So where p value is greater
t_test_equal_var_p_ls = df[df['p-value'] <= 0.05] # So where p value is less

# 11/14/2023 t-test for equal variances for 2020 stops here
*Null hypothesis is that the means are the same. And alternative is that they differ. so if p value is low, you reject the null hypothesis. Since p value is not significant, the filtered filtered_df_p_values in the above dataframe are those which there mean for each test value is not higher. 

## 11/14/2023
T test for unequal variances now for 2020



In [ ]:
# First get dataframe values in a list
prints=abcd[['Measure','State1','State2']]
n = len(prints)
current = prints.values.tolist()


In [ ]:
x=[]
y=[]
n = len(prints)
for i in range(n):  # For i in range n, set the current list parts to separate variables.
	current1 = current[i]
	measure=(current1[0])

	state1=(current1[1]) # first second, third element
	state2=(current1[2].strip()) # Need to strip random whitesapce


	condition1 = s2020.loc[s2020['StateAbbr']==state1]
	#print(len(condition1))
	lis1 = condition1.loc[condition1['Measure']==measure,'Data_Value'] # For a dataframe .loc seems to be the simplest for performing this work
	condition2 = s2020.loc[s2020['StateAbbr']==state2]
	#print(len(condition2))
	lis2 = condition2.loc[condition2['Measure']==measure,'Data_Value']
	y.append([state1,state2])
	if len(lis1) != len(lis2):
		min_len = (min(len(lis1), len(lis2)))
		
		if len(lis1) == min_len:

			lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
			
		if len(lis2) == min_len:
			lis1= random.sample(list(lis1), min_len)
		#print(len(lis1),len(lis2))
	#print(stats.ttest_ind(lis1, lis2,equal_var=True,alternative='greater'),i)
	x.append([state1,state2,measure,stats.ttest_ind(lis1, lis2,equal_var=False,alternative='greater'),i]) # Make sure to set the alternative to greater
	print_count += 1
	#print("Total prints:", print_count)
	#result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together

In [ ]:
df=pd.DataFrame(x)
df['result_string'] = df[3].astype(str)
df['t-statistic'] = df['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Extract out all of the relevant p values and t statistics
df['p-value'] = df['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
df['State1']= df[0]
df['state2']=df[1]
df= df.drop(['result_string',0,1,3,4],axis=1) 

In [ ]:
t_test_unequal_var_p_gr = df[df['p-value'] > 0.05] # So where p value is greater
t_test_unequal_var_p_ls = df[df['p-value'] <= 0.05] # So where p value is less

In [ ]:
t_test_unequal_var_p_gr.head(1)

In [ ]:
t_test_unequal_var_p_ls.head(1)

## 11/14/2023 Need to Go back and fix this. 
Where means are not different. 

To visuaklize this maybe.. group the counts? And put it in a bar chart like the one above so the format is the similiar. 

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].count()
city_counts1 = city_counts1.sort_values(ascending=False)

In [ ]:
city_counts1.info()

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].count()
city_counts1 = city_counts1.sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts1,palette="Spectral")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")

# 11/14/2023
For t-values data visualizations maybe count where p value is higher for say texas. Then count where it is not, and maybe display by state? 

Okay so maybe if texas p value is higher (so a low p value) add some number to texas
Otherwise add the number to the other state

# First get dataframe values in a list
state_counts = []
df['combined_text']=df['State1']+df['state2']
prints=df[['p-value','State1','state2','combined_text']]

n = len(prints)
current = prints.values.tolist()
count1 = 0
count2 = 0
x=[]
y=[]
n = len(prints)
for i in range(n):  # For i in range n, set the current list parts to separate variables.
	current1 = current[i]
	measure=(current1[0])

	state1=(current1[1]) # first second, third element
	state2=(current1[2].strip()) # Need to strip random whitesapce
	state_counts.append([state1,state2])
	set_of_tuples = {tuple(inner_list) for inner_list in state_counts} # Create a set of tuples now after all of the appending is done.
	listnow=list(set_of_tuples) # list of all of my variables
	list_of_dicts = [{state: 0 for state in ls} for ls in listnow] # Because I like doing things inefficiently. Back into a dictionary now.

	
			

## 11/14/2023
The below code needs to compare what is in result list to what is in current1

# Now that the list is there maybe iterate again?
n = len(prints)
current1 = prints.values.tolist()
for i in range(n):  # For i in range n, set the current list parts to separate variables.
	current1 = current[i]
	measure=current1[0]

	state1=current1[1] # first second, third element
	state2=current1[2].strip() # Need to strip random whitesapce
	combos=current1[3]
	result_list = [''.join(sublist) for sublist in listnow]

## 11/14/2023 USE THIS
Simpler Is better. So just loc the result dataframe and do the lengths of each. 
The code right below takes the length of the state names list and does.. something. \
I guess sorts and counts and so I have to append those counts to some kind of list. 

But now I can visualize the data properly. By taking everything out of the set of tuples and making a bar graph like the one for the original data

## TO DO 11/15/2023 The below is for unequal var
visualize the p values for texas in front of (or behind) every other state
 

In [ ]:
prints=df['state2']  # get the state all by itself
prints=prints.unique() 
lists1=[]
for i in prints:   
	abc = df.loc[df['state2']==i]
	abcd = abc[abc['p-value'] < 0.05]
	defg = abc[abc['p-value'] > 0.05]
	lists1.append(['TX',len(abcd),i,len(defg)])
	set_of_tuples = {tuple(inner_list) for inner_list in lists1}


In [ ]:
df10 = pd.DataFrame(set_of_tuples)
df10['combined'] = df10[0]+'/'+df10[2]

In [ ]:
df10.head(2)

In [ ]:
X_1 = range(len(set_of_tuples))
x_val= [x[0] for x in set_of_tuples]
y_val = [x[1] for x in set_of_tuples]
x_val1 = [x[2] for x in set_of_tuples]
y_val1 =[x[3] for x in set_of_tuples]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(data=(y_val),alpha=0.9)
ax = sns.barplot(data=y_val1, alpha=0.7)
ax.set_xticklabels(df10['combined'], rotation='vertical', fontsize=10)
ax.set(xlabel='States',
       ylabel='count of p_values that are significant per state VS Texas',
       title='Resulting P values'
       )
ax.text(2, 27, " Texas = ♦", 
       fontsize = 12,          # Size
       fontstyle = "oblique",  # Style
       color = "blue",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(4, 26,"All other States = ♦", 
       fontsize = 12,          # Size
       fontstyle = "oblique",  # Style
       color = "orange",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 

# looking at the dataframe above 

In [ ]:
#

# 11/15/2023 
Looking over the data, when compared about half of the values have significane. 

## Code right above is the list organization code. 

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].count()
city_counts1 = city_counts1.sort_values(ascending=False)
city_counts2 = s2021.groupby('StateAbbr')['LocationName'].count()
city_counts2 = city_counts2.sort_values(ascending=False)
city_counts3 = s2022.groupby('StateAbbr')['LocationName'].count()
city_counts3 = city_counts3.sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts1,palette="Spectral")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")

In [ ]:
city_counts2 = filtered_df_p_values1.groupby('State1')[2].count()
city_counts2 = city_counts2.sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts2,palette="Spectral")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")

# The above is all well and good. (11/10/2023)
But it still does not answer my question. Just states that they are different. 
So I am going to attempt a T test on Monday. 


In [ ]:
x=[]
y=[]
n = len(dataframes_list2020)
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df
	print(subseta)
	matching_indices1 = [index for index, df in enumerate(dataframes_list2021) if (df['StateAbbr'].isin(subseta).any())]# No matter where they are, this could should find the matching states. 
	for index in matching_indices1:
		matches_2021 = dataframes_list2021[index]
	matching_indices2 = [index for index, df in enumerate(dataframes_list2022) if (df['StateAbbr'].isin(subseta).any())]
	for index in matching_indices2:
		matches_2022 = dataframes_list2022[index]
		subset1=(matches_2020['Measure'].unique()) 
		subset2=(matches_2021['Measure'].unique())  # Now collect the unique measures from all df
		subset3=(matches_2022['Measure'].unique())
		intersection_of_measures = set(subset1).intersection(subset2).intersection(subset3) # now the intersection of all 3
		intersection_list = list(intersection_of_measures)
		for i in intersection_list:
			lis1 = matches_2020.loc[matches_2020['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
			lis2 = matches_2021.loc[matches_2021['Measure']==i,'Data_Value']
			lis3 = matches_2022.loc[matches_2022['Measure']==i,'Data_Value'] # Now to deal with if they are not the same length
			if len(lis1) != len(lis2) != len(lis3):  # If the lengths of the lists are not equal I will have to deal with that
					min_len = (min(len(lis1), len(lis2),len(lis3))) # Set the minimum length
					if len(lis1) == min_len:
						lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
						lis3= random.sample(list(lis3), min_len) # Random sample does not use replacement
					if len(lis2) == min_len:
						lis1= random.sample(list(lis1), min_len)
						lis3= random.sample(list(lis3), min_len)
					if len(lis3) == min_len:
						lis1= random.sample(list(lis1), min_len)
						lis2= random.sample(list(lis2), min_len)
					else:
						continue
			print(stats.kruskal(lis1, lis2, lis3),i)
			x.append([subseta,stats.kruskal(lis1, lis2, lis3),i])
			print_count += 1
			print("Total prints:", print_count)
	

## Working down here for correlations as of 11/15/2023

Trying to organize my values and decide if the values should be dropped. The below code filtered out all of the combinations that had corr values that were not significant. As far as how to move forward. Not sure

In [ ]:
df1

In [ ]:
# I had mixed data in the column so I needed to weirdly split this thing up 
def split_column(row):  
    return pd.Series([row[0][0], row[1]])  
df1[['state', 'Corr']] = df1[0].apply(split_column)
df1.drop(0, axis=1, inplace=True) # Drop the df in place. dont assign to another var
df1[['Measure1','Measure2']] = df1[1].apply(lambda x: pd.Series(x)) # This splits the string up for us and such
empty_cells = df1.isna() # Are there any na values
empty_cells_count = empty_cells.sum()
print(empty_cells_count)
empty_rows = df1[df1.isna().any(axis=1)]
df2 = df1[~df1.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
df2.drop(1,axis=1, inplace=True)
len(df2.Corr.unique())


In [ ]:
abcd = df2[df2['Corr'] < 0.5]
defg=abcd[abcd['Corr'] >-0.5]
aa=defg[defg['state']=='NC']
len(aa['Measure1'].unique())

In [ ]:
city_counts1 =defg.groupby('state')['Measure1'].nunique()

In [ ]:
cleaned_df = df2[~df2.index.isin(defg.index)]

In [ ]:
city_counts2 =cleaned_df.groupby('state')['Measure1'].nunique()

In [ ]:
ah = df2[df2['Corr'] > 0.5]

In [ ]:
len(ah['Measure1'].unique())

In [ ]:
ab=df2[df2['state']=='NC']

In [ ]:
len(ab)

In [ ]:
len(defg)

In [ ]:
len(cleaned_df)

In [ ]:
len(df2)

In [ ]:
# Correlation values are good

defg['Measure2'].value_counts() # Sorting in place

In [ ]:

df3=df2.sort_values(by=['Corr'])
df3.sort_values(['Measure1', 'Measure2'], inplace=True) # Obtaining measures

In [ ]:
mean_values = df3['Corr'].mean

In [ ]:
value = df2.loc[5283, 'Corr']

In [ ]:
df3['Combined'] = df3['Measure1'].astype(str) + '_' + df3['Measure2'].astype(str) # Combining strings

In [ ]:
grouped_df = df3.groupby('Combined')

In [ ]:
mean_values = grouped_df['Corr'].median()

In [ ]:
mean_values

In [ ]:
mean_values_reset = mean_values.reset_index()
mean_values_reset

In [ ]:
mean_values_reset.info()

In [ ]:
filtered_series = mean_values[(mean_values < 0.5) & (mean_values > -0.5)] # Filter all of the values. 

In [ ]:
filtered_series=filtered_series.reset_index()

In [ ]:
filtered_series.info()

In [ ]:
filtered_series


In [ ]:
stuff_to_drop=(mean_values_reset[~mean_values_reset.Combined.isin(filtered_series.Combined)])

In [ ]:
print(stuff_to_drop['Combined'])

In [ ]:
stuff_to_keep=(df3[~df3.Combined.isin(stuff_to_drop.Combined)])

In [ ]:
stuff_to_keep['Combined'].nunique()

In [ ]:
grouped = stuff_to_keep.groupby('state') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')